### Optimal Portfolio Allocation

An investment universe of the following risky assets with a dependence structure (correlation) is given:

\begin{equation\*}

\begin{matrix}
\textbf{Asset} & \boldsymbol\mu & \boldsymbol\sigma & \boldsymbol\omega\\
A & 0.02 & 0.05 & \omega_1\\
B & 0.07 & 0.12 & \omega_2\\
C & 0.15 & 0.17 & \omega_3\\
D & 0.20 & 0.25 & \omega_4
\end{matrix}
\qquad
\qquad
R =
\begin{pmatrix}
1 & 0.3 & 0.3 & 0.3\\
0.3 & 1 & 0.6 & 0.6\\
0.3 & 0.6 & 1 & 0.6\\
0.3 & 0.6 & 0.6 & 1
\end{pmatrix}

\end{equation\*}


**Question 1.** Consider minimum variance portfolio with a target return m.

\begin{equation*}
\underset{\boldsymbol\omega}{\arg\min} \frac{1}{2}\omega'\Sigma\omega
\qquad
s.t.
\quad
\omega'1 = 1
\quad
\mu\_\Pi = \omega'\mu = m
\end{equation*}

- Formulate the Lagrangian and give its partial derivatives.
- Write down the analytical solution for optimal allocations $ \omega^\* $ (derivation not required).
- Inverse optimisation: generate above 700 random allocation sets (vectors) 4 × 1, those will not be optimal allocations.
  - Standardise each set to satisfy $\omega'1 = 1$, in fact you can generate 3 allocations and compute the 4th.
  - For each vector of allocations compute $\mu_\Pi = \omega'\mu$ and $\sigma_\Pi = \sqrt{\omega'\Sigma\omega}$.
  - Plot the cloud of points of $\mu_\Pi$ vertically on $\sigma_\Pi$ horizontally. <ins>Explain this plot</ins>.

Hint: Since we treat this as an inverse optimisation, there is no computation of $\omega^*$ from the ready formula.


### Answers and code


Form the Langrange function with two multipliers $\lambda$ and $\gamma$ for a given vector of weights $w$
\begin{equation*}
L(w, \lambda, \gamma) = \frac{1}{2}w'\Sigma w + \lambda(m - w'\mu) + \gamma(1 - w'1)
\end{equation*}

where $(m - w'\mu)$ is the return constraint and $(1 - w'1)$ is the budget constraint.

##### First order condition

The optimal portfolio is the point where the gradient of $L(w, \lambda, \gamma) = 0$.

Therefore $\frac{\partial L}{\partial w} = 0$.

\begin{equation*}
\frac{\partial L}{\partial w} = \frac{1}{2} 2 \Sigma w + \lambda (-\mu) + \gamma(-1) = \Sigma w - \lambda \mu - \gamma 1 = 0
\end{equation*}

##### Second order condition

Given this is a minimisation problem, we seek the Hessian to be greater than 1

\begin{equation*}
\frac{\partial^2 L}{\partial w^2} = \Sigma
\end{equation*}

As teh covariance matrix is positive, the 2nd order condition is satisfied.

Therefore the optimal allocation $w^*$ can be derived

\begin{equation*}
\Sigma w - \lambda \mu - \gamma 1 = 0 \newline
\end{equation*}

\begin{equation*}
w^* = \Sigma^{-1} (\lambda \mu + \gamma 1)
\end{equation\*}

Substituting the above optimal weight $w^*$ into the constraint

\begin{equation*}
\mu'w = \mu' \Sigma^{-1} (\lambda \mu + \gamma 1) = \mu' \Sigma^{-1} \lambda \mu + \mu' \Sigma^{-1}\gamma 1 = m
\end{equation*}
and
\begin{equation*}
1'w = 1' \Sigma^{-1} (\lambda \mu + \gamma 1) = 1' \Sigma^{-1} \lambda \mu + 1' \Sigma^{-1}\gamma 1 = 1
\end{equation*}

Setting scalars for ease of presentation and computation
\begin{equation*}
A = 1\Sigma^{-1} 1'
\end{equation*}

\begin{equation*}
B = \mu' \Sigma^{-1} 1 = 1' \Sigma^{-1} \mu
\end{equation*}

\begin{equation*}
C = \mu' \Sigma^{-1} \mu
\end{equation*}

The constraint functions then become

\begin{equation*}
C \lambda + B \gamma = m
\end{equation*}

\begin{equation*}
B \lambda + A \gamma = 1
\end{equation*}

This is a system of 2 equations with 2 unknowns

\begin{equation*}
\lambda = \frac{m - B \gamma}{C}
\end{equation*}

therefore

\begin{equation*}
\frac{Bm - B^2 \gamma}{C} + A \gamma = 1
\end{equation*}

\begin{equation*}
Bm - B^2 \gamma + A C \gamma = C
\end{equation*}

\begin{equation*}
A C \gamma - B^2 \gamma = C - Bm
\end{equation*}

\begin{equation*}
\gamma = \frac{C - Bm}{AC - B^2}
\end{equation*}

in the same manner

\begin{equation*}
\lambda = \frac{Am - B}{AC - B^2}
\end{equation*}

Finally, substituting back into optimal weight $w^*$

\begin{equation*}
w^* = \Sigma^{-1} (\lambda \mu + \gamma 1) = \frac{1}{AC - B^2} \Sigma^{-1}[(A \mu - B1)m + (C1 - B \mu)]
\end{equation\*}


In [269]:
# Import the relevant packages
# Import numpy
import plotly.express as px
import numpy as np
from numpy import *
from numpy.linalg import multi_dot

# Import pandas
import pandas as pd

# Import cufflinks
import cufflinks as cf
cf.set_config_file(offline=True, dimensions=((800, 500)))

# Import plotly express for EF plot
# px.defaults.template = "plotly_dark"
px.defaults.width, px.defaults.height = 800, 500


In [270]:
# Set the matrices for mu and sigma
mu = np.array([0.02, 0.07, 0.15, 0.2]).reshape((4, 1))
sigma = np.array([0.05, 0.12, 0.17, 0.25]).reshape((4, 1))
R = np.array([[1, 0.3, 0.3, 0.3], [0.3, 1, 0.6, 0.6], [
             0.3, 0.6, 1, 0.6], [0.3, 0.6, 0.6, 1]]).reshape(4, 4)

# Compute the covariance matrix usings SRS where S = the diagonal standard deviation matrix
S = np.diag(sigma.flatten())
covariance_matrix = multi_dot([S, R, S])

# Set values for N and initiate an empty list for the
N = 10000
weights_arr = []

# Generate N random weights for the 4 assets
for i in range(N):
    weights = random.random(4)
    # Standardise these weights
    weights /= sum(weights)
    weights_arr.append(weights)

# Add them to a dataframe
df = pd.DataFrame(weights_arr, columns=['w_1', 'w_2', 'w_3', 'w_4'])


In [271]:
df.head()


,w_1,w_2,w_3,w_4
0,0.213665,0.217394,0.439406,0.129535
1,0.264130,0.246068,0.176436,0.313366
2,0.402716,0.034662,0.529080,0.033542
3,0.319958,0.147541,0.344863,0.187638
4,0.409675,0.558014,0.022816,0.009495


In [272]:
df.tail()


,w_1,w_2,w_3,w_4
9995,0.315941,0.348620,0.207883,0.127556
9996,0.438047,0.057066,0.014181,0.490707
9997,0.323807,0.299285,0.040825,0.336083
9998,0.317320,0.062259,0.287280,0.333142
9999,0.301478,0.170056,0.156352,0.372113


In [273]:
# loop through weights, extract matrix, calculate portfolio mean and sd, append back to df
for index, row in df.iterrows():
    # convert to a matrix of weights
    w = np.array(row).reshape(1, 4)
    # Compute mean for the portfolio, add it to the dataframe
    mu_pi = multi_dot([w, mu])
    df.loc[index, 'mu_pi'] = mu_pi.flatten()
    # Compute the sd for the portfolio, add it to the dataframe
    sigma_pi = np.sqrt(multi_dot([w, covariance_matrix, w.T]))
    df.loc[index, 'sigma_pi'] = sigma_pi.flatten()


In [274]:
df.head()


,w_1,w_2,w_3,w_4,mu_pi,sigma_pi
0,0.213665,0.217394,0.439406,0.129535,0.111309,0.120549
1,0.264130,0.246068,0.176436,0.313366,0.111646,0.125834
2,0.402716,0.034662,0.529080,0.033542,0.096551,0.105910
3,0.319958,0.147541,0.344863,0.187638,0.105984,0.113721
4,0.409675,0.558014,0.022816,0.009495,0.052576,0.079608


In [275]:
df.tail()


,w_1,w_2,w_3,w_4,mu_pi,sigma_pi
9995,0.315941,0.348620,0.207883,0.127556,0.087416,0.100133
9996,0.438047,0.057066,0.014181,0.490707,0.113024,0.136743
9997,0.323807,0.299285,0.040825,0.336083,0.100767,0.120466
9998,0.317320,0.062259,0.287280,0.333142,0.120425,0.130474
9999,0.301478,0.170056,0.156352,0.372113,0.115809,0.131023


In [277]:
# Plotting the results
fig = px.scatter(df, x='sigma_pi', y='mu_pi', color='mu_pi', color_continuous_scale='purp',
                 labels={'sigma_pi': 'Expected Volatility',
                         'mu_pi': 'Expected Return'},
                 title='Efficient Frontier showing highlighting maximum and minimum return')
idx_max_mu = df['mu_pi'].idxmax()
idx_min_mu = df['mu_pi'].idxmin()

fig.add_scatter(x=[df['sigma_pi'].loc[idx_max_mu]], y=[df['mu_pi'].loc[idx_max_mu]], marker=dict(color='green', size=10, symbol='cross'),
                name='Maximum Return').update(layout_showlegend=False)
fig.add_scatter(x=[df['sigma_pi'].loc[idx_min_mu]], y=[df['mu_pi'].loc[idx_min_mu]], marker=dict(color='red', size=10, symbol='cross'),
                name='Minimum Return').update(layout_showlegend=False)
fig.show()


The above plot builds up an example through simulated data showing the efficient frontier of a portfolio of 4 risky assets. 